In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import os
import cv2

In [ ]:
trainpath = '/kaggle/input/plant-pathology-2021-fgvc8/train.csv'
labelpath = 'label.csv' # not yet exists
imgfolder = '/kaggle/input/plant-pathology-2021-fgvc8/train_images'
dataset = pd.read_csv(trainpath)
len(dataset)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
train_df, val_df = train_test_split(dataset)

In [ ]:
len(train_df), len(val_df)

In [ ]:
train_df.head()

In [ ]:
train_img_Path = '../input/resized-plant2021/img_sz_256'
# test_img_Path = '../input/plant-pathology-2021-fgvc8/test_images'
CLASSES = train_df['labels'].unique().tolist()


In [ ]:
len(os.listdir(train_img_Path))

In [ ]:
train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
train_data = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            directory=train_img_Path,
            x_col="image",
            y_col="labels",
            target_size=(150, 150),
            batch_size=32,
            class_mode='categorical')
val_data = train_datagen.flow_from_dataframe(
            dataframe=val_df,
            directory=train_img_Path,
            x_col="image",
            y_col="labels",
            target_size=(150, 150),
            batch_size=32,
            class_mode='categorical')

In [ ]:
from keras.applications import ResNet101V2, ResNet152V2, InceptionResNetV2, Xception
from keras import Sequential
from tensorflow.keras.layers import Conv2D, Dropout,MaxPooling2D,Flatten,Dense
base_ResNet152V2 = ResNet152V2(include_top = False, 
                         weights = '../input/keras-pretrained-models/ResNet152V2_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)

In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_ResNet2 = Sequential()
model_ResNet2.add(base_ResNet152V2)
model_ResNet2.add(Dense(12, activation=('softmax')))

model_ResNet2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_ResNet2.summary()

# Training the CNN on the Train data and evaluating it on the val data
b = model_ResNet2.fit(train_data, validation_data = val_data, epochs = 10, batch_size=128)

In [ ]:
def load_model():
    from keras import models, layers, losses,optimizers
    model = models.Sequential()
    model.add(layers.Conv2D(120, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(90, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(72, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dense(12))
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss=losses.BinaryCrossentropy())
    model.summary()
    return model

model = load_model()
r = model.fit(train_data, validation_data =val_data, epochs = 10, batch_size=128)